## How cinema seats are reserved ?

In this notebook, our goal is to collect data on the reservation of seats in cinemas

We will use a new API exposed by UGC France

We will check the seats status every minute for several hours

In [ ]:
import requests
import json
import csv
import datetime
import time
import string

url = "https://xmlresa.ugc.fr/numerotation/api/plan/033/46004431"
headers = {}


startTime = time.time()
while True:
    currentTime = datetime.datetime.now()
    currentTimeFormatted = currentTime.strftime("%Y-%m-%d_%H%M%S")
    fileName = "seats_" + currentTimeFormatted + ".csv"
    
    response = requests.request("GET", url, headers= headers)
    if response.status_code == 200 :
        responseJson = json.loads(response.text)
        with open(fileName, 'w', newline='', encoding='UTF8') as f:
            writer = csv.writer(f, delimiter=';')
            header = ["rangee","siege", "statut"]
            writer.writerow(header)
    
            sieges = responseJson["sieges"]
            for thisSiege in sieges :
                rangee = thisSiege["rangee"]
                siege = thisSiege["siege"]
                statut = thisSiege["statut"]
                rowData = [rangee, siege, statut]
                writer.writerow(rowData)
            
    time.sleep(60.0 - ((time.time() - startTime) % 60.0))

## Making a timelaps

We have collected more than 180 images and will use them to create a stunning timelapse.

In [37]:
import pandas as pd
df = pd.read_csv("data/seats_2023-03-25_182827.csv", sep=";")
df

,rangee,siege,statut
0,A,0,L
1,A,14,L
2,A,13,L
3,A,12,L
4,A,11,L
...,...,...,...
171,K,4,L
172,K,3,L
173,K,2,L
174,K,0,L


## Converting the data in the files to images representing seats in the cinema

In [41]:
from os import listdir
from os.path import isfile, join

imagesPath = "images/"
dataPath = "data/"

def convertDataToImage(fileName):
    df = pd.read_csv(dataPath + fileName, sep=";")
    y = df["rangee"].unique()
    x = df["siege"].unique()
    width = len(x)
    height = len(y)
    realWidth = width * 20
    realHeight = height * 20
    image = Image.new('RGB', (realWidth, realHeight))
    
    availableColor = (0, 255, 0)
    occupiedColor = (255, 0, 0)

    statutColorDict = {"L": availableColor,
                       "O": occupiedColor
                       }

    for i in range(len(df)):
        rangee = df.loc[i, "rangee"]
        siege = df.loc[i, "siege"]
        statut = df.loc[i, "statut"]
        xSeat = string.ascii_lowercase.index(rangee.lower())
        ySeat = siege
        realX = 20 * xSeat
        realY = 20 * ySeat
        for i in range(1, 20):
            for j in range(1, 20):
                image.putpixel((realY + j, realX + i), statutColorDict[statut])

    flippedImage = image.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
    rotatedflippedImage = flippedImage.transpose(Image.Transpose.ROTATE_90)
    rotatedflippedImage = rotatedflippedImage.transpose(Image.Transpose.ROTATE_90)

    rotatedflippedImage.save(imagesPath + fileName.split(".csv")[0] + ".png")




csvFiles = [f for f in listdir(dataPath) if isfile(join(dataPath, f))]
for thisFile in csvFiles :
    convertDataToImage(thisFile)

## Generating a gif from the images : 10 fps

In [35]:
import imageio.v2 as imageio
imagesPath = "images/"

imagesFileNames = [f for f in listdir(imagesPath) if isfile(join(imagesPath, f))]
images = []
for imageFileName in imagesFileNames:
    images.append(imageio.imread("images/"+imageFileName))
imageio.mimsave('movie.gif', images, fps = 10)

In [42]:
from IPython.display import Image
from IPython.core.display import HTML
HTML('<img src="movie.gif">')
display(HTML('<img src="movie.gif">'))